In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9GAMMA_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a = load_pickle(get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX))
# Load sample B
docs_sample_b = load_pickle(get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX))

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load corpus

In [9]:
doc_directory = load_pickle(get_relative_path(data_path, DOC_DIRECTORY))
# Corpus - tokens
corpus_bag_of_words = load_pickle(get_relative_path(data_path, CORPUS_BAG_OF_WORDS))
corpus_tokens = load_pickle(get_relative_path(data_path, CORPUS_TOKENS))
dictionary_tokens = load_pickle(get_relative_path(data_path, DICTIONARY_TOKENS))
# Corpus - keyphrases
corpus_bag_of_keyphrases = load_pickle(get_relative_path(data_path, CORPUS_BAG_OF_KEYPHRASES))
dictionary_keyphrases = load_pickle(get_relative_path(data_path, DICTIONARY_KEYPHRASES))

print(len(doc_directory))
print("Test:", doc_directory[docs_sample_a[0]])
print("Test:", doc_directory[docs_sample_b[0]])

# print("Test:", corpus_bag_of_words[doc_directory[docs_sample_a[0]]])
# print("Test:", corpus_bag_of_words[doc_directory[docs_sample_b[0]]])
print(len(corpus_bag_of_words), len(dictionary_tokens))
print(len(corpus_bag_of_keyphrases), len(dictionary_keyphrases))

10000
Test: 2111
Test: 483
10000 62388
10000 107954


In [5]:
corpus_a_tokens = [corpus_bag_of_words[doc_directory[docid]] for docid in docs_sample_a]
corpus_b_tokens = [corpus_bag_of_words[doc_directory[docid]] for docid in docs_sample_b]

corpus_a_keyphrases = [corpus_bag_of_keyphrases[doc_directory[docid]] for docid in docs_sample_a]
corpus_b_keyphrases = [corpus_bag_of_keyphrases[doc_directory[docid]] for docid in docs_sample_b]

print(len(corpus_a_tokens), len(corpus_b_tokens), len(corpus_a_keyphrases), len(corpus_b_keyphrases))

5000 5000 5000 5000


In [6]:
import numpy as np
from gensim import models
from gensim.similarities import Similarity
from gensim.test.utils import get_tmpfile
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## TFIDF

In [7]:
model_tf_idf_tokens = models.TfidfModel(corpus_bag_of_words)
model_tf_idf_keyphrases = models.TfidfModel(corpus_bag_of_keyphrases)

2018-09-19 11:27:05,163 : INFO : collecting document frequencies
2018-09-19 11:27:05,165 : INFO : PROGRESS: processing document #0
2018-09-19 11:27:05,509 : INFO : calculating IDF weights for 10000 documents and 62387 features (988950 matrix non-zeros)
2018-09-19 11:27:05,690 : INFO : collecting document frequencies
2018-09-19 11:27:05,691 : INFO : PROGRESS: processing document #0
2018-09-19 11:27:05,784 : INFO : calculating IDF weights for 10000 documents and 107953 features (232171 matrix non-zeros)


Example

In [11]:
doc = corpus_bag_of_words[doc_directory[docs_sample_a[0]]]
vector1 = model_tf_idf_tokens[doc]

doc = corpus_bag_of_keyphrases[doc_directory[docs_sample_a[0]]]
vector2 = model_tf_idf_keyphrases[doc]

# Example
print(vector1[:5])
print(vector2[:5])

print(corpus_b_tokens[0])
print(corpus_tokens[0])

[(16, 0.012841377027749524), (36, 0.008727667495260607), (50, 0.028852844980048267), (51, 0.005664177941739807), (80, 0.054138131117306684)]
[(105, 0.06695925194076603), (470, 0.07503314787978065), (507, 0.13897025827840198), (566, 0.06990219084546122), (1274, 0.05774539521913921)]
[(3, 1), (24, 1), (33, 1), (69, 1), (130, 1), (131, 4), (148, 1), (167, 1), (189, 1), (192, 1), (213, 1), (232, 1), (247, 1), (254, 4), (264, 2), (287, 1), (289, 1), (316, 1), (356, 1), (360, 1), (395, 1), (410, 2), (424, 1), (457, 1), (523, 1), (542, 1), (600, 1), (609, 1), (624, 2), (658, 1), (666, 1), (711, 1), (827, 1), (862, 1), (878, 1), (954, 9), (963, 2), (964, 1), (975, 1), (977, 3), (1031, 1), (1154, 1), (1201, 1), (1271, 1), (1394, 1), (1431, 1), (1519, 1), (1556, 1), (1611, 1), (1628, 1), (1663, 4), (1748, 4), (1777, 1), (1819, 1), (1831, 1), (1861, 1), (1867, 1), (2038, 1), (2110, 1), (2113, 1), (2114, 1), (2119, 2), (2269, 3), (2378, 1), (2381, 1), (2579, 1), (2586, 1), (2605, 1), (2735, 2), (2

In [10]:
from scipy.spatial.distance import cosine as cosine_distance
mat = np.zeros((len(docs_sample_a), len(docs_sample_b)))

In [ ]:
for i, id_a in enumerate(docs_sample_a):
    doc_index = doc_directory[id_a]
    a = set(corpus_tokens[doc_index])
    for j, id_b in enumerate(docs_sample_b):
        doc_index = doc_directory[id_b]
        b = corpus_tokens[doc_index]
        sd = a.symmetric_difference(b)
        
        mat[i][j] = 1.0 - cosine_distance(v1, v2)
mat.shape

In [ ]:
index_tf_tokens = Similarity(get_tmpfile("index-tf-tokens"), corpus_b_tokens, num_features=len(dictionary_tokens))

save_pickle(np.array(index_tf_tokens[corpus_a_tokens]), get_relative_path(data_path, SIM_TF_TOKENS))

index_tf_keyphrases = Similarity(get_tmpfile("index-tf-keyphrases"), corpus_b_keyphrases, num_features=len(dictionary_keyphrases))
save_pickle(np.array(index_tf_keyphrases[corpus_a_keyphrases]), get_relative_path(data_path, SIM_TF_KEYPHRASES))

In [ ]:
index_tf_idf_tokens = Similarity(get_tmpfile("index-tf-idf-tokens"), model_tf_idf_tokens[corpus_b_tokens], num_features=len(dictionary_tokens))
save_pickle(np.array(index_tf_idf_tokens[model_tf_idf_tokens[corpus_a_tokens]]), get_relative_path(data_path, SIM_TF_IDF_TOKENS))

index_tf_idf_keyphrases = Similarity(get_tmpfile("index-tf-idf-keyphrases"), model_tf_idf_keyphrases[corpus_b_keyphrases], num_features=len(dictionary_keyphrases))
save_pickle(np.array(index_tf_idf_keyphrases[model_tf_idf_keyphrases[corpus_a_keyphrases]]), get_relative_path(data_path, SIM_TF_IDF_KEYPHRASES))